<a href="https://colab.research.google.com/github/saraMuay/face-liveness/blob/main/extract_frame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import io
import os
import numpy as np
import base64
from IPython.display import HTML
import cv2
#from google.colab.patches import cv2_imshow
from tensorflow.keras.models import model_from_json
from PIL import Image
#import glob
import shutil

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!cp -r "/content/gdrive/MyDrive/antispoofing.zip" "/content"

In [5]:
import zipfile
archive = zipfile.ZipFile('/content/gdrive/MyDrive/antispoofing.zip')
archive.extractall('/content')

In [6]:
root_dir =  '/content/antispoofing'

In [27]:
!ls antispoofing

face_antispoofing.ipynb  model


In [7]:
face_cascade = cv2.CascadeClassifier("/content/antispoofing/model/haarcascade_frontalface_default.xml")

In [8]:
json_file = open("/content/antispoofing/model/finalyearproject_antispoofing_model_mobilenet.json",'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights('/content/antispoofing/model/┘Çantispoofing_model.h5')

In [9]:
def playvideo(filename):
    video = io.open(filename, 'r+b').read()
    encoded = base64.b64encode(video)
    return HTML(data='''<video alt="test" controls>
                    <source src="data:video/mp4;base64,{0}" type="video/mp4"/>
                 </video>'''.format(encoded.decode('ascii')))

In [11]:
video = 'WhatsApp Video 2021-10-13 at 9.36.54 PM.mp4'
playvideo(video)

In [12]:
cap = cv2.VideoCapture(video)

In [13]:
save_path = os.mkdir('/content/save')

In [14]:
i=0 
gap =10

while True:
    try:
      ret,frame = cap.read()
      if i==0:
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray,1.3,5)
      if i%gap==0:
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray,1.3,5)

        i+=1
        print("{} Found {0} Faces.".format(i,len(faces)))

      

      for (x,y,w,h) in faces:
        face = frame[y-5:y+h+5,x-5:x+w+5] 
        resized_face = cv2.resize(face,(160,160))
        resized_face = resized_face.astype("float") / 255.0
        resized_face = np.expand_dims(resized_face, axis=0)


        preds = model.predict(resized_face)[0]
        print(preds)
        if preds> 0.5:
          label = 'Spoof'
          cv2.putText(frame, label, (x,y - 10),
          cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)   
          cv2.rectangle(frame, (x, y), (x+w,y+h),(0, 0, 255), 2)
        else:
          label = 'Real'
          cv2.putText(frame, label, (x,y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
          cv2.rectangle(frame, (x, y), (x+w,y+h),(0, 255, 0), 2)
        
        print("[INFO] Object found. Saving locally.")
        print(label)
        cv2.imwrite(str(w) + str(h) +label+ '_faces.jpg', frame)
        cv2.imwrite(f"{save_path}/{i}.png", frame)
        
        status = cv2.imwrite(str(predect)+'faces_detected.jpg', frame)
        print("[INFO] Image faces_detected.jpg written to filesystem: ", status) 

        key = cv2.waitKey(1)
        if key == ord('q'):
          break
    except Exception as e:
        pass
video.release()        
cv2.destroyAllWindows()

[INFO] Found 1 Faces.
[3.704127e-05]
[INFO] Object found. Saving locally.
real
[INFO] Found 1 Faces.
[4.0023784e-05]
[INFO] Object found. Saving locally.
real
[INFO] Found 1 Faces.
[2.5632107e-05]
[INFO] Object found. Saving locally.
real
[INFO] Found 1 Faces.
[0.00013873]
[INFO] Object found. Saving locally.
real
[INFO] Found 1 Faces.
[5.8330144e-05]
[INFO] Object found. Saving locally.
real
[INFO] Found 1 Faces.
[9.384727e-05]
[INFO] Object found. Saving locally.
real
[INFO] Found 1 Faces.
[2.7034794e-05]
[INFO] Object found. Saving locally.
real
[INFO] Found 1 Faces.
[2.130959e-05]
[INFO] Object found. Saving locally.
real
[INFO] Found 1 Faces.
[4.8771577e-05]
[INFO] Object found. Saving locally.
real
[INFO] Found 1 Faces.
[4.0199255e-05]
[INFO] Object found. Saving locally.
real
[INFO] Found 1 Faces.
[3.33425e-05]
[INFO] Object found. Saving locally.
real
[INFO] Found 1 Faces.
[4.02195e-05]
[INFO] Object found. Saving locally.
real
[INFO] Found 1 Faces.
[0.00012021]
[INFO] Object f

KeyboardInterrupt: ignored

In [23]:
frame_path = os.mkdir('/content/frames')

In [41]:
real_frame_path = os.mkdir('/content/frames/real')

In [44]:
spoof_frame_path= os.mkdir('/content/frames/spoof')

In [ ]:
#!dir
all_file= os.listdir()

all_file

In [64]:
#PATH = all_file
#Copy_path="/content/frames"

for filename in all_file:
  if filename[-14:].lower() == 'real_faces.jpg':
    each_frame_path = os.path.join('content', filename)
    print(each_frame_path)
    if os.path.isfile(each_frame_path):
      shutil.copyfile(each_frame_path, real_frame_path)
  
  elif filename[-15:].lower() == 'spoof_faces.jpg':
    each_frame_path = os.path.join('content', filename)
    print(each_frame_path)
    if os.path.isfile(each_frame_path):
      shutil.copyfile(each_frame_path, spoof_frame_path)

content/163163real_faces.jpg
content/101101real_faces.jpg
content/158158real_faces.jpg
content/105105real_faces.jpg
content/5050spoof_faces.jpg
content/110110real_faces.jpg
content/5555spoof_faces.jpg
content/2626spoof_faces.jpg
content/166166real_faces.jpg
content/159159real_faces.jpg
content/113113real_faces.jpg
content/5353spoof_faces.jpg
content/114114real_faces.jpg
content/152152real_faces.jpg
content/150150real_faces.jpg
content/156156real_faces.jpg
content/4747spoof_faces.jpg
content/116116real_faces.jpg
content/5252real_faces.jpg
content/5454spoof_faces.jpg
content/2727spoof_faces.jpg
content/160160real_faces.jpg
content/4848spoof_faces.jpg
content/119119real_faces.jpg
content/5151spoof_faces.jpg
content/165165real_faces.jpg
content/4747real_faces.jpg
content/4848real_faces.jpg
content/5151real_faces.jpg
content/5252spoof_faces.jpg
content/162162real_faces.jpg
content/9999real_faces.jpg
content/164164real_faces.jpg
content/132132spoof_faces.jpg
content/5353real_faces.jpg
conten